In [1]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(os.path.join(project_root, 'code'))

from model.dataset import MIDD
from model.run_train import run
import model.config as config

import warnings 
warnings.filterwarnings("ignore")

import torch
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, classification_report
from transformers import AutoTokenizer
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import time
from pathlib import Path


/home/zhhaibo2023/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 训练模型

In [2]:
SEED = 2025
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [3]:
device = 'cuda:1'

# 加载数据
r_data = torch.load('../data/midd.pt')
tokenizer = AutoTokenizer.from_pretrained(config.ptm)  # 使用BERT中文分词器

dataset_name = 'midd'
dataset = MIDD(
    dataset_name = dataset_name,
    response_texts=r_data['responses'],
    labels=r_data['labels'],
    tokenizer=tokenizer,
    max_length=64
)
print(len(dataset))



520


In [4]:
LR = 1e-5
Weight_Decay = 1e-4
EPOCHS = 50
BS = 32
Dropout = 0.5


gammas = [1.0, 0.7072, 0.2005]


classify_flag ='bin' # penta

logdir_root = os.path.join(os.getcwd(), dataset_name)
print(logdir_root)

/home/zhhaibo2023/program/msce/code/run_model/midd


In [5]:
test_label_list_perfold, test_pred_list_perfold, test_id_list_perfold, best_test_acc, best_test_fs = run(dataset=dataset,
                                                                                                        dataset_name=dataset_name,
                                                                                                        logdir_root=logdir_root,
                                                                                                        classify_flag=classify_flag,
                                                                                                        device=device,
                                                                                                        gammas=gammas,
                                                                                                        SEED=SEED,
                                                                                                        BS=BS,
                                                                                                        LR=LR,
                                                                                                        Weight_Decay=Weight_Decay,
                                                                                                        Dropout=Dropout,
                                                                                                        EPOCHS=EPOCHS,
                                                                                                        K=5)


bert_model_path: /home/zhhaibo2023/Pretrained_Model/bert-base-chinese


Model: DepressionDetectionModel
num_stimuls:  9
fold: 1, epoch: 1, train_loss: 3.1093, train_acc: 54.33, train_fscore: 67.35, test_loss: 2.3679, test_acc: 72.12, test_fscore: 80.0, time: 9.2 sec
fold: 1, epoch: 2, train_loss: 2.2887, train_acc: 55.29, train_fscore: 66.06, test_loss: 1.7911, test_acc: 59.62, test_fscore: 74.7, time: 7.55 sec
fold: 1, epoch: 3, train_loss: 1.9026, train_acc: 64.42, train_fscore: 73.38, test_loss: 1.6077, test_acc: 82.69, test_fscore: 86.96, time: 7.62 sec
fold: 1, epoch: 4, train_loss: 1.7443, train_acc: 69.23, train_fscore: 76.73, test_loss: 1.5796, test_acc: 65.38, test_fscore: 62.5, time: 7.58 sec
fold: 1, epoch: 5, train_loss: 1.6154, train_acc: 78.12, train_fscore: 81.91, test_loss: 1.3568, test_acc: 88.46, test_fscore: 90.32, time: 7.85 sec
fold: 1, epoch: 6, train_loss: 1.5046, train_acc: 83.89, train_fscore: 86.13, test_loss: 1.2874, test_acc: 86.54, test_fscore: 88.52, time: 7.46 sec
fold: 1, epoch: 7, train_loss: 1.4536, train_acc: 82.21, train